In [2]:
import requests
import folium
import heapq
from folium import Circle, PolyLine

user_location = (30.269745410969797, 77.99290170226186)


In [3]:
help_stations_with_phone = [
    {"coordinates": (30.2837296, 77.997745), "phone_number": "+919389969916"},
    {"coordinates": (30.304785, 78.0209032), "phone_number": "+919389969916"},
    {"coordinates": (30.2909374, 78.0498482), "phone_number": "+919389969916"},
    {"coordinates": (30.3108936, 78.0121377), "phone_number": "+919389969916"},
    {"coordinates": (30.2820388, 78.002518), "phone_number": "+919389969916"},
    {"coordinates": (30.2875988, 78.0231428), "phone_number": "+919389969916"},
    {"coordinates": (30.2789269, 78.0012372), "phone_number": "+919389969916"},
    {"coordinates": (30.3125064, 78.0219506), "phone_number": "+919389969916"},
    {"coordinates": (30.3029695, 78.0214584), "phone_number": "+919389969916"},
    {"coordinates": (30.2919587, 77.9986905), "phone_number": "+919389969916"},
    {"coordinates": (30.311503, 77.982168), "phone_number": "+919389969916"},
    {"coordinates": (30.2864273, 78.0480676), "phone_number": "+919389969916"},
    {"coordinates": (30.3070576, 78.0095762), "phone_number": "+919389969916"}
]


help_stations = [station["coordinates"] for station in help_stations_with_phone]
phone = []

## Dijkstra's Algorithm

In [4]:
def get_osrm_distance(origin, destination):
    url = f"http://router.project-osrm.org/route/v1/foot/{origin[1]},{origin[0]};{destination[1]},{destination[0]}"
    params = {"overview": "full", "geometries": "geojson"}
    response = requests.get(url, params=params)
    data = response.json()
    if 'routes' in data and len(data['routes']) > 0:
        distance_km = data['routes'][0]['distance'] / 1000
        coords = [(coord[1], coord[0]) for coord in data['routes'][0]['geometry']['coordinates']]
        return distance_km, coords
    else:
        return float('inf'), []


In [5]:
def dijkstra(graph, start):
    pq = [(0, start)]
    distances = {start: 0}
    previous_nodes = {start: None}

    while pq:
        current_distance, current_node = heapq.heappop(pq)

        if current_distance > distances.get(current_node, float('inf')):
            continue

        for neighbor, weight in graph.get(current_node, {}).items():
            distance = current_distance + weight
            if distance < distances.get(neighbor, float('inf')):
                distances[neighbor] = distance
                previous_nodes[neighbor] = current_node
                heapq.heappush(pq, (distance, neighbor))

    return distances, previous_nodes


In [6]:
graph = {}
for station in help_stations:
    graph[station] = {}
    for other_station in help_stations:
        if station != other_station:
            dist, _ = get_osrm_distance(station, other_station)
            graph[station][other_station] = dist

graph[user_location] = {}
for station in help_stations:
    dist, _ = get_osrm_distance(user_location, station)
    graph[user_location][station] = dist


In [7]:
distances, _ = dijkstra(graph, user_location)
help_distances = [(station, distances[station]) for station in help_stations if station in distances]
nearest = sorted(help_distances, key=lambda x: x[1])[:3]

for station, _ in nearest:
    for station_info in help_stations_with_phone:
        if station_info["coordinates"] == station:
            phone.append(station_info["phone_number"])

print("\n3 Nearest Help Stations (by Dijkstra walking distance):")
for i, (station, dist) in enumerate(nearest):
    print(f"{i+1}. Location: {station}, Distance: {dist:.2f} km, Phone: {phone[i]}")



3 Nearest Help Stations (by Dijkstra walking distance):
1. Location: (30.2789269, 78.0012372), Distance: 1.81 km, Phone: +919389969916
2. Location: (30.2820388, 78.002518), Distance: 2.13 km, Phone: +919389969916
3. Location: (30.2837296, 77.997745), Distance: 2.66 km, Phone: +919389969916


In [8]:
m = folium.Map(location=user_location, zoom_start=15)
folium.Marker(user_location, tooltip="User", icon=folium.Icon(color='blue')).add_to(m)
Circle(user_location, radius=6000, color="blue", fill=False).add_to(m)

for station in help_stations:
    folium.Marker(station, tooltip="Help Station", icon=folium.Icon(color="gray")).add_to(m)

for i, (station, dist) in enumerate(nearest):
    folium.Marker(
        station,
        tooltip=f"Help Station {i+1} ({dist:.2f} km), Phone: {phone[i]}",
        icon=folium.Icon(color="green")
    ).add_to(m)

    _, coords = get_osrm_distance(user_location, station)
    PolyLine(coords, color='green', weight=4, opacity=0.7).add_to(m)

m.save("live_osrm_nearest_help_stations_with_phone.html")
print("Map saved as 'live_osrm_nearest_help_stations_with_phone.html'")
m


Map saved as 'live_osrm_nearest_help_stations_with_phone.html'


In [9]:
print(phone)

['+919389969916', '+919389969916', '+919389969916']


In [10]:
from twilio.rest import Client

account_sid = 'AC4c024fc964c6236c3b439abd84d49329'
auth_token = '946268ec7561cd7ea890a635da8e2c8d'

from_phone = '+19137330629'
message = "Help "

client = Client(account_sid, auth_token)
for to_phone in phone:
    try:
        message_sent = client.messages.create(
            body=message,
            from_=from_phone,
            to=to_phone
        )
        print(f"Message sent to {to_phone}: {message_sent.sid}")
    except Exception as e:
        print(f"Failed to send message to {to_phone}: {e}")

Message sent to +919389969916: SM89fba98681b679a2f73c8e730ec37a22
Message sent to +919389969916: SM14e335ba6a92c4a21b29eab8101f252f
Message sent to +919389969916: SMb468f958a0b5023eb0993a0ce9550a96
